In [ ]:
import sys
sys.path.append('../')

### Run the optimization

In [ ]:
import os

import numpy as np
from common import get_clustered_dataframes, evaluate_best_coin, objective

import mlflow
import optuna

os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5000"
mlflow.set_experiment('Training_LSTM_RNN_Cluster_SMA')

# Normalize the data to the range [0, 1] to help the LSTM model converge faster

clusters_data = get_clustered_dataframes(add_sma_columns=True)

for cluster, df in clusters_data.items():

    print(f"Cluster {cluster}")
    criptos = df.filter(like='close', axis=1).columns
    output_shape = len(criptos)
    criptos = ','.join([cripto.split("_")[1] for cripto in criptos])

    df = df.drop('Date', axis=1)
    data = np.array(df)

    storage_name = "sqlite:///training_cluster_sma.db"
    pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
    study = optuna.create_study(study_name=criptos, direction='minimize', storage=storage_name, pruner=pruner)

    # Start the optimization process
    study.optimize(lambda trial: objective(trial, data, criptos, output_shape), n_trials=50)

    # Get the best hyperparameters
    try:
        best_params = study.best_params
        print("Best Hyperparameters:", best_params)
        evaluate_best_coin(criptos, data, best_params, output_shape)
    except ValueError:
        print("No best hyperparameters found")